In [4]:
from src.data.loader import load_datasets
import pandas as pd

In [3]:
customers, transactions = load_datasets()

/app/src/data/loader.py:15: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions = pd.read_csv(TRANSACTIONS_RAW)


In [8]:
transactions.head()

,WeekDay,Channel,cust_id,D_C,Currency_trx,Amount_EUR,Customer_IBAN,Counterparty_IBAN,Amount_Orig,currency,MCC,country_of_merchant,BookingDatetime
0,5,POS,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,D,EUR,7.518036,3a8c99d1e8caf60ddbc4b9865b0947f981c90980f3fab2...,7ecc6d620a9ea4170231178c236206770eb7a4785d2a6c...,7.518036,EUR,5271.0,EE,2023-01-27 10:00:28
1,2,Internet Bank,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,D,EUR,0.200481,3a8c99d1e8caf60ddbc4b9865b0947f981c90980f3fab2...,7ecc6d620a9ea4170231178c236206770eb7a4785d2a6c...,0.200481,EUR,NaN,NaN,2023-01-17 02:14:14
2,3,Internet Bank,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,D,EUR,1.002405,3a8c99d1e8caf60ddbc4b9865b0947f981c90980f3fab2...,7ecc6d620a9ea4170231178c236206770eb7a4785d2a6c...,1.002405,EUR,NaN,NaN,2023-02-08 02:00:18
3,7,Internet Bank,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,C,EUR,0.010024,3a8c99d1e8caf60ddbc4b9865b0947f981c90980f3fab2...,3a8c99d1e8caf60ddbc4b9865b0947f981c90980f3fab2...,0.010024,EUR,NaN,NaN,2023-01-01 03:56:39
4,3,Internet Bank,00feb0ff373287a3b1b210369f5aef9bfffd5d02f6bc8f...,D,EUR,1.002405,3a8c99d1e8caf60ddbc4b9865b0947f981c90980f3fab2...,7ecc6d620a9ea4170231178c236206770eb7a4785d2a6c...,1.002405,EUR,NaN,NaN,2023-03-08 02:06:48


In [9]:
customers.head()

,cust_id,customer_type,parent_company_flag,CUST_GRP_ID,BRTH_DT,language
0,3caf82febb1d4e64b140893d9e89d748115ec5b70455e1...,SME,0,eefcfda39744d0f4c7d1f1f6f13a53dbf49f7281de5de0...,2005-2009,ENG
1,fd0d32ecf697980ff5b750d18fe6f8e9f96f2e31fba130...,SME,1,NaN,2015-2019,EST
2,320b10457c9c109c2b4f00b0fecac1cc8cb7c2da8ad936...,SME,1,NaN,2020-2024,EST
3,d4d7e033ed758bb075ad96435d4afb505b7a4059aa3f37...,SME,1,NaN,2020-2024,EST
4,e9b8374b1f3ae04cc65672bc5907726d962edb35e05736...,SME,1,NaN,2020-2024,EST
